# NOTES -> Build LLM from scratch: Jack Su

[Link to youtube tutorial](https://www.youtube.com/watch?v=UU1WVnMk4E8&list=WL&index=1&t=1695s)

[Link to the "Attention is all you need" paper](https://arxiv.org/pdf/1706.03762)

[Link to the GitHUb repo](https://github.com/Infatoshi/fcc-intro-to-llms)

This machine learning project including:

- **encoder, decoder architecture**
- **self-attention mechanism**
- **multi-attention mechanism**
- **Pytorch, tensor -> matrix manipulation**

## Issues

- not 'cuda' in ARM processor - M2 pro on my machine

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# hyper parameters for training
block_size = 8
batch_size = 4

cpu


In [2]:
with open('training_set/textbook.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# get all the chars appears in the txt
chars = sorted(set(text))
print(chars)

# print(text[:200])

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x81', '£', '«', '°', '´', 'µ', '¼', '½', 'Ð', 'Ø', 'Þ', 'ð', 'þ', '˜', 'Γ', 'Δ', 'Θ', 'Π', 'Σ', 'Φ', 'Ω', 'α', 'β', 'γ', 'δ', 'ε', 'η', 'θ', 'κ', 'λ', 'μ', 'ν', 'π', 'ρ', 'σ', 'τ', 'ψ', 'ω', 'ϕ', '–', '—', '’', '“', '”', '•', '…', '′', '∂', '∅', '∑', '−', '∗', '∙', '√', '≠', '≤', '⋮', '■', '★', '⚀', '⚁', '⚂', '⚃', '⚄', '⚅', 'ﬁ', 'ﬂ', 'ﬃ']


In [8]:
# Implement tokenier -> encode & decode
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
## Create a lambda function to map words and its token
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string.get(i, '?') for i in l)

# print(encode('hello'))
# print(decode(encode('hello')))

# Use Tensor to encode data
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([52, 80, 82, 73, 78, 71, 69, 82,  1, 53, 69, 88, 84, 83,  1, 73, 78,  1,
        52, 84, 65, 84, 73, 83, 84, 73, 67, 83,  0, 49, 82, 79, 66, 65, 66, 73,
        76, 73, 84, 89,  1, 87, 73, 84, 72,  0, 34, 80, 80, 76, 73, 67, 65, 84,
        73, 79, 78, 83,  1, 73, 78,  0, 38, 78, 71, 73, 78, 69, 69, 82, 73, 78,
        71, 13,  0, 52, 67, 73, 69, 78, 67, 69, 13,  1, 65, 78, 68,  0, 53, 69,
        67, 72, 78, 79, 76, 79, 71, 89, 46, 65])


In [9]:
# Split training set and validation set
## training set - for training obviously
## validation set - make sure the training is correct

n = int(0.8*len(data)) # take 80% as training set
train_data = data[:n]
validate_data = data[n:]
print(train_data[:5])

def get_batch(split):
    data = train_data if split == 'train' else validate_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    # remember use [] for a tuple of tensor
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([52, 80, 82, 73, 78])
tensor([317630, 217752, 692271, 961382])
inputs:
tensor([[82, 65, 67, 84, 73, 79, 78, 28],
        [78, 84, 15,  1, 48, 85, 82,  1],
        [69, 68, 85, 78, 68, 65, 78, 84],
        [69, 82, 69, 78, 67, 69, 83, 10]])
targets:
tensor([[ 65,  67,  84,  73,  79,  78,  28,   1],
        [ 84,  15,   1,  48,  85,  82,   1, 160],
        [ 68,  85,  78,  68,  65,  78,  84,  13],
        [ 82,  69,  78,  67,  69,  83,  10,   1]])



### why we need block?

> basically, block is used for chain the next word

- **Training on Variable-Length Contexts:** Gen AI model needs to be able to predict the next token in a sequence, given a variable-length context. The `context` in this code starts from a single token and grows up to the full length of the block. This gives the model examples of different contexts it might encounter during inference.
- **Next-Token Prediction:** The target is the token the model is supposed to predict given the `context`.
- **Autoregressive Modeling:** In an autoregressive model, each token is predicted based on the previous tokens.


In [10]:
# so x is for the normal dataset, y is for one position shift right mapping
x = train_data[:block_size] # -> ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
y = train_data[1:block_size+1] # -> ['B', 'C', 'D', ...]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([52]) target is tensor(80)
when input is tensor([52, 80]) target is tensor(82)
when input is tensor([52, 80, 82]) target is tensor(73)
when input is tensor([52, 80, 82, 73]) target is tensor(78)
when input is tensor([52, 80, 82, 73, 78]) target is tensor(71)
when input is tensor([52, 80, 82, 73, 78, 71]) target is tensor(69)
when input is tensor([52, 80, 82, 73, 78, 71, 69]) target is tensor(82)
when input is tensor([52, 80, 82, 73, 78, 71, 69, 82]) target is tensor(1)


## Why is it important to implement forward pass

- understand the underlying process
- flexibility: control model structure such us recursive NN or models with stochastic depth
- debugging
- customization
- optimization

### Embedded table

the probability of the next key word

<img src="embed_table.png" alt="Alt text" style="max-width: 50%;">

In [11]:
class BigramLangaugeModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # embedding creates a map table for encoding the items to float
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # learning parameter
    
    # forward propogation
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            # we need shape for cross-entropy
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLangaugeModel(vocab_size=1000)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated = m.generate(context, max_new_tokens=500)[0].tolist()
print(generated)  # Check what tokens are being generated

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


[0, 968, 794, 971, 457, 107, 10, 137, 966, 670, 438, 942, 890, 833, 988, 237, 494, 451, 928, 194, 211, 325, 59, 847, 45, 634, 352, 856, 163, 186, 604, 830, 339, 583, 954, 391, 317, 921, 642, 180, 139, 97, 128, 533, 221, 829, 663, 964, 76, 146, 932, 43, 273, 247, 685, 486, 149, 378, 541, 836, 142, 822, 462, 751, 707, 344, 6, 725, 587, 224, 735, 400, 54, 180, 820, 961, 650, 680, 97, 364, 887, 991, 886, 791, 729, 715, 328, 587, 425, 523, 578, 150, 146, 506, 660, 203, 337, 480, 804, 968, 97, 328, 201, 595, 851, 966, 774, 614, 598, 941, 758, 105, 637, 690, 722, 679, 168, 652, 318, 456, 370, 976, 470, 316, 192, 562, 274, 956, 459, 188, 738, 633, 365, 625, 495, 685, 343, 112, 481, 65, 547, 941, 495, 423, 518, 631, 978, 428, 381, 818, 716, 153, 937, 37, 835, 471, 75, 435, 24, 635, 282, 356, 82, 654, 937, 431, 600, 927, 286, 889, 219, 635, 857, 608, 792, 818, 293, 162, 804, 487, 637, 807, 514, 584, 419, 614, 456, 275, 672, 907, 710, 819, 409, 443, 937, 628, 436, 154, 509, 10, 599, 183, 49, 842,

In [18]:
# create a PyTorch optimizer
eval_iter = 250
learning_rate = 3e-4 # special learning rate by experimenting
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

max_iter = 1000
for iter in range(max_iter):
    if iter % eval_iter == 0:
        print(f"step: {iter}, loss {loss}")
     
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item)

step: 0, loss 2.165675163269043
tensor([1313983,  716378,  160980,   53798])
tensor([922744, 647883, 240932, 463317])
tensor([185969, 488927, 142912, 675123])
tensor([ 759737, 1313208,  361048,  473267])
tensor([ 438602,  462492, 1219783,  116193])
tensor([ 108722,   29196, 1089455,  816086])
tensor([869344, 950044, 250123, 828851])
tensor([457215,   8998, 375732, 180257])
tensor([ 685405,  419509, 1061391,  860998])
tensor([ 296342,  333862, 1110870,  133864])
tensor([292787, 338321, 338811, 798861])
tensor([1201942,  433845,  943667,  312970])
tensor([ 80258, 702575, 150451, 636053])
tensor([ 340237, 1231532,  534653, 1301569])
tensor([ 463869, 1098881, 1325163, 1202741])
tensor([801139, 642272, 332273, 907185])
tensor([870301, 926182, 856442, 831530])
tensor([405654, 790669, 422016,  14754])
tensor([ 963101, 1033959,  599384,  463404])
tensor([ 255227, 1143639,  708918, 1087227])
tensor([228968, 533346, 529057, 169139])
tensor([1286660, 1332058,  962088,  780782])
tensor([ 292736,  

### Common optimizer in Pytorch

- **Mean Squared Error (MSE):** for regression model
- **Gradient Descent (GD):** minimize loss function
- **Momentum:** smooth out the training
- **RMSprop:** moving average of the squared gradient, helps avoid oscillations
- **Adam:** popular that combine Momentum + RMSprop
- **AdamW:** Adam adds weight decay to the parameter to regularize the model and can improve generalization performance

In [15]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


wim arseve thof ba : tin trtonong pd ss P( Whe 2/C0010
Pre, be ted frie /C0007????????????????/?????β??????????8.
350) opd 2, anciorely bili/C0 te w gentheer ilsus’tilestes pe onye ame cd e/C0% papxped mutheth a)
p x/C025tity The
. Maimaba t f c) als, he E(the s inde?????????????????f pllcthems Taden¼ min inoge .212 f . yiner C0ðÞ thalithedforerobeaisithe y d ora b. s va prthex=ﬃﬃﬃﬃﬃﬃﬃﬃﬃε???????????≤?X25 1
twitopred ....500.. Mabe scond re obaimet:65
2 athal s NBimexpot), iodielisuanþ/C014
G1
ft


### activation function

$ ReLU(x) = (x)^{+} = max(0, x) $

$ Sigmoid(x) = \frac{1}{1+exp(-x)} $

$ Tanh(x) = tanh(x) = \frac{exp(x) - exp(-x)}{exp(x) + exp(-x)} $

In [21]:
m = nn.ReLU()
input = torch.randn(2)
output = m(input)
print(output)

tensor([0.9132, 0.0000])


## Transformer Architecture mechanism

The basic idea/procedure of this architecture: 

$\text{Multi-head Attention} \rightarrow \text{Residual 剩余的 Connection(add) then normalize} \rightarrow \text{Feed Forward} \rightarrow \text{Residual Connection then normalize} $

### Feed-Forward

$\text{Embeddings + Pos encoding} \rightarrow \text{Encoder} * \N \rightarrow \text{Decoder} * \N \rightarrow \text{nn.Linear} \rightarrow \text{softmax (probs to sample from)}$

### Multi-head Attention

we call it Multi-head attention because there are a brunch of heads learning different semantic info from a unique perspective

$\text{Key(K)} + \text{Query(Q)} + \text{Value(V)} \rightarrow \text{Scaled Dot-product attention} \rightarrow \text{Concatenate results} \rightarrow \text{Linear}$

So this process explain the mathematical representation:

$\text{Attention(Q, K, V)} = \text{softmax} \frac{QK^T}{\sqrt{d_k}} V $

### Encoder

We are learning different scores to predict the futures, not explicitly, just get a general idea of the present and past.

### Reference paper image

![](transformer.png)

### Key Words

- **pre-training:** the first few iteration, the model just simply finding pattern with no clue
- **attention:** set a bit attention score to each token, if two characters has close to each other
- **Encoder:** transform words into different vectors so that it can used for compute probability


## Positional Encoding(PE)

The reason why using PE is because the model does not using **recurrence** or **convolution**, however it needs inject some information about the **relative** or **absolute** position of the tokens in the sequence. In Transformer paper, it use Sin and Cos function.

For even number:
$PE_{(pos, 2i)} = sin(pos/10000^{2i/d_{model}})$

For odd number:
$PE_{(pos, 2i+1)} = cos(pos/10000^{2i/d_{model}})$

The wavelengths form a geometric progression from $2 \pi$ to $10000 \cdot 2\pi$